preprocess.py (전체 흐름)

load & reorient & HU

resample 3D

skull strip & optional center crop

window & normalize (loop over experiments)

pad_or_crop_safe to shape.volume

for each axis, method → generate projection, apply same window/normalize, pad2d to shape.slice

torch.save with keys

In [ ]:
# 로컬: %cd C:\Users\401-09\Preprocessing

In [ ]:
# 필요시 폴더 삭제
!rm -r 경로명

In [ ]:
%cd /content

/content


In [1]:
# GitHub에서 프로젝트 클론
#    → <YOUR_REPO_URL> 부분을 실제 리포지토리 URL로 바꿔주세요.
!git clone https://github.com/MANDUnoko/Preprocessing
%cd Preprocessing

Cloning into 'Preprocessing'...
remote: Enumerating objects: 646, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 646 (delta 69), reused 108 (delta 39), pack-reused 498 (from 1)
Receiving objects: 100% (646/646), 572.75 MiB | 23.31 MiB/s, done.
Resolving deltas: 100% (256/256), done.
/content/Preprocessing


In [2]:
# Google Drive 마운트 (필요한 경우)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
!git fetch origin main|

/bin/bash: -c: line 2: syntax error: unexpected end of file


In [31]:
!git checkout origin/main -- /content/Preprocessing/scripts/preprocess.py

In [32]:
!git checkout origin/main -- /content/Preprocessing/configs/preprocessing.yaml

In [3]:
!sed -n '1,20p' configs/preprocessing.yaml

# 기본 데이터 경로
data_dir: "data"

# 파일 확장자
extensions:
  raw:         ".nii.gz"
  brain_mask:  ".nii.gz"
  lesion:      ".nii"

# Voxel spacing (z, y, x) — 원본 스캔 vs 목표 isotropic
spacings:
  original: [5.0, 0.4121, 0.4121]  # 원본 CT의 Z/Y/X 해상도 [mm]
  target:   [5.0, 1.0, 1.0]           # 재샘플링

# 실험할 Window 범위 목록 (clip_min, clip_max)
window:
  experiments:
    - [0, 150]

# 전처리 결과물(.pt)에는 모든 실험 채널이 쌓이고, 시각화나 모델 학습 시에는 experiments 중 원하는 채널을 선택


In [4]:
%run scripts/preprocess.py --data_dir "/content/drive/MyDrive/project/2차 시도/data" --all


>>> Processing case 124
Skull-strip 옵션 비활성화됨 → 원본 볼륨 사용
[DEBUG] shape after axis standardization: (15, 211, 512)
[DEBUG] vol_s 통계
  min: -1021.7999877929688, max: 2800.36669921875, mean: -537.564697265625
원본 HU 범위: -1022.0 2848.0
Resampled 후 HU 범위: -1021.8 2800.3667
Skullstrip 후 HU 범위: -1021.8 2800.3667
Mask sum: 0.0 Brain mask sum: 1593593.0
volume_channels[0] mean/std: 0.158200 0.308493
[Signal Quality]
  Lesion voxels:     0
  Background voxels: 17920000
  SNR:  0.0000  (기준 > 1.0)
  CNR:  0.0000  (기준 > 0.5)
[✔] Preprocessed and saved: 124.pt

>>> Processing case 112
Skull-strip 옵션 비활성화됨 → 원본 볼륨 사용
[DEBUG] shape after axis standardization: (16, 211, 512)
[DEBUG] vol_s 통계
  min: -1013.6666870117188, max: 2415.13330078125, mean: -593.8687744140625
원본 HU 범위: -1016.0 2625.0
Resampled 후 HU 범위: -1013.6667 2415.1333
Skullstrip 후 HU 범위: -1013.6667 2415.1333
Mask sum: 0.0 Brain mask sum: 1996040.0
volume_channels[0] mean/std: 0.147460 0.287331
[Signal Quality]
  Lesion voxels:     0
  Backgr

In [ ]:
from pathlib import Path
import torch
import numpy as np

# ===== 사용자 설정 =====
case_id = "049"
processed_dir = Path("/content/drive/MyDrive/project/2차 시도/data/processed")
pt_path = processed_dir / f"{case_id}.pt"

# ===== .pt 로드 및 분해 =====
data = torch.load(pt_path)
image = data["volume"][0].numpy()     # shape: (D, H, W)
mask  = data["mask"][0].numpy()

# ===== 기본 통계 =====
print(f"[Image Stats: case {case_id}]")
print(f"  Mean:     {image.mean():.4f}")
print(f"  Std:      {image.std():.4f}")
print(f"  Min/Max:  {image.min():.2f} / {image.max():.2f}")
print(f"  Shape:    {image.shape}")
print(f"  Spacing:  {data['meta']['spacing']}")

# ===== Signal Quality (SNR / CNR) =====
lesion     = image[mask == 1]
background = image[(mask == 0) & (image > 0)]

if lesion.size > 0 and background.size > 0:
    snr = lesion.mean() / (lesion.std() + 1e-5)
    cnr = abs(lesion.mean() - background.mean()) / (background.std() + 1e-5)
    status = "Good" if snr > 1.0 and cnr > 0.5 else "Poor"

    print("\n[Signal Quality]")
    print(f"  Lesion voxels:     {lesion.size}")
    print(f"  Background voxels: {background.size}")
    print(f"  SNR:  {snr:.4f}  (기준 > 1.0)")
    print(f"  CNR:  {cnr:.4f}  (기준 > 0.5)")
    print(f"  상태 판단: {status}")
else:
    print("\n[Signal Quality] 병변 또는 배경 데이터가 충분치 않음")

[Image Stats: case 049]
  Mean:     0.3563
  Std:      0.4523
  Min/Max:  0.00 / 1.00
  Shape:    (140, 250, 512)
  Spacing:  (5.0, 1.0, 1.0)

[Signal Quality]
  Lesion voxels:     8919
  Background voxels: 7829978
  SNR:  2.7187  (기준 > 1.0)
  CNR:  0.5047  (기준 > 0.5)
  상태 판단: Good


In [ ]:
import torch
import nibabel as nib
import numpy as np
from pathlib import Path

# 1. .pt 파일 로드
case_id = "049"
pt_path = Path("/content/drive/MyDrive/Daejeon_Medcoding(2024-12~2025-07)/이미지 프로젝트/2차 시도/data/processed") / f"{case_id}.pt"
data    = torch.load(pt_path)

vol   = data["volume"].numpy()    # shape = (C, D, H, W)
mask  = data["mask"].numpy()      # shape = (1, D, H, W) or (D, H, W)

# 2. 채널 중 하나 골라서 (D, H, W) 배열 획득
vol0 = vol[0]                     # (D, H, W)
mask = np.squeeze(mask)           # (D, H, W)

# 3. 화면용으로 transpose: (D, H, W) → (H, W, D)
vol_view  = np.transpose(vol0, (1, 2, 0))   # (H, W, D)
mask_view = np.transpose(mask, (1, 2, 0))   # (H, W, D)

# 4. affine 설정: 여기선 단위행렬 사용
affine = np.eye(4)

# 5. NIfTI 생성 & 저장
out_dir = Path("/content/drive/MyDrive/Daejeon_Medcoding(2024-12~2025-07)/이미지 프로젝트/2차 시도/data")
out_dir.mkdir(parents=True, exist_ok=True)

nib.save(nib.Nifti1Image(vol_view.astype(np.float32), affine),
         out_dir / f"{case_id}_vol_view.nii.gz")
nib.save(nib.Nifti1Image(mask_view.astype(np.uint8), affine),
         out_dir / f"{case_id}_mask_view.nii.gz")

print(f"[✔] View용 NIfTI 저장 완료: {case_id}")

[✔] View용 NIfTI 저장 완료: 049
